<img src="cryotempo_logo.png" alt="logo" width="200"/> <img src="esa_logo.png" alt="esa" width="170"/> <img src="earthwave_logo.png" alt="earthwave" width="150"/> <img src="UoE_logo.png" alt="uoe" width="200"/>

##<strong>*This is a Jupyter notebook that demonstrates how to download and use Cryotempo-EOLIS data, downloaded from Specklia. Here, we will investigate the coverage and quality of the point product*</strong> 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


To run this notebook, you will need to make sure that the folllowing packages are installed in your python environment (all can be installed via pip/conda).

    - matplotlib: for plotting
    - datetime: for handling timestamps
    - numpy
    - specklia: for data retrieval

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
!pip install contextily
!pip install specklia

###<strong>1) Download Data

Regardless of whether you are using the Google Colab environment, or have downloaded this notebook to your local drive, you will first need to download some data. You can use this notebook to plot any CryoTEMPO-EOLIS point data that you choose. For a quick example, follow the below instructions to download a small example dataset.</strong>

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Imports necessary to run the rest of the code

In [32]:
import contextily as ctx
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from specklia import Specklia
from shapely import Polygon
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.rcParams.update({'font.size': 16})

<strong>The following cells download point product data for the Jakobshavn glacier from Specklia, Earthwave's geospatial point cloud database.</strong>

First, we will instantiate our Specklia Python client. To run the code, you will need to generate your own Specklia API key.

In [33]:
user_api_key = input('Please generate your own key using https://specklia.earthwave.co.uk/ApiKeys and paste it here:')
specklia_client = Specklia(user_api_key)

Next, we define our area of interest and query Specklia for data:

In [ ]:
wider_jakobshavn_area_polygon = Polygon(([-49.97, 68.92], [-48.24, 68.96], [-48.38, 69.86], [-50.19, 69.81], [-49.97, 68.92]))

dataset_name = 'CryoTEMPO-EOLIS Point Product'
available_datasets = specklia_client.list_datasets()
point_product_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

point_product_data, sources = specklia_client.query_dataset(
    dataset_id=point_product_dataset['dataset_id'],
    epsg4326_polygon=wider_jakobshavn_area_polygon,
    min_datetime=datetime(2023,1,1),
    max_datetime=datetime(2023,3,1))

print(
    f'Query complete, {len(point_product_data)} points returned, drawn from {len(sources)} original sources.')


Now, let's explore the retrieved data. First, we can investigate the metadata of the data sources. In the output of this command, we can see lots of additional information associated with the first source, including projection information, product version and support information. This is the same information that would be included in the header of the source NetCDF file!

In [ ]:
sources[0]

For the EOLIS datasets, the source information will also include the ESA FTP server path of the corresponding stripfile:

In [ ]:
for i, source in enumerate(sources):
    print(f'ESA FTP server path of source #{i+1} is: {source["source_information"]["science_pds_path"]}')

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
###<strong>2. Investigate the point product</strong>

<strong>Now that the data has been loaded into the notebook, we can use it to investigate the coverage and quality of the CryoTEMPO-EOLIS point prodcut over the Jakobshavn glacier. Let's create some visuals to quickly understand the scope of the point product and its capabilities.</strong>

If we run the cell below, we can see that the variables available for analysis include a time, x and y position, elevation and an associated uncertainty value. For more information about the derivation of these variables, see the CryoTEMPO-EOLIS ATBD, available at https://cryotempo-eolis.org/product-description/

In [ ]:
print('Variables returned by query: ', point_product_data.columns)

We can use the source information to determine the geospatial projection of the x and y columns:

In [ ]:
projections = []
for i, source in enumerate(sources):
    projection_for_source = source["source_information"]["geospatial_projection"]
    print(f'X and Y columns in source # {i+1} are in projection: {projection_for_source}')
    projections.append(projection_for_source)

if len(set(projections)) == 1:
    print('All the sources use the same projection! We will need this later on.')
    geospatial_projection = projections[0]
else:
    raise ValueError('You queried an area that covers multiple EOLIS regions - the X and Y columns in data returned by your query are not in the same projection. Please run a different query or reproject before continuing!')

Next, we make a simple spatial plot of the point product that we have downloaded. This plot shows all point data available for the Jakobshavn glacier in the timeframe that we specified for the query, and demonstrates the spatial coverage achieved using swath processing. In the Figure on the left, the scatter points are colour-coded by their elevations. On the right, the colour demonstrates the uncertainty associated with each point. For more information on the observing tracks of CryoSat and swath processing, see https://earth.esa.int/eogateway/news/cryosats-swath-processing-technique

We plot a background map using contextily, this may take a couple of minutes as the background imagery is downloaded.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,15))
query_results_sample = point_product_data.sample(1000)

for i, column in enumerate(['elevation', 'uncertainty']):
    c = axes[i].scatter(point_product_data['x'], point_product_data['y'], c=point_product_data[column], s=1, cmap='viridis')
    plt.colorbar(c, ax=axes[i], label=f'{column.title()} [m]')

    axes[i].set_xlabel('x [m]')
    axes[i].set_ylabel('y [m]')
    plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    
    ctx.add_basemap(axes[i], source=ctx.providers.GeoportailFrance.orthos, zoom=8, crs=geospatial_projection)

    # add thumbnail to corner of plot to show where the data is
    thumbnail_ax = inset_axes(axes[i], width="20%", height=3, loc='upper left')

    # set extent of greenland for thumbnail axis in EPSG:3413
    thumbnail_ax.set_ylim(-3400000, -700000)
    thumbnail_ax.set_xlim(-700000, 900000)
    thumbnail_ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
    for spine in ['bottom', 'top', 'right', 'left']:
        thumbnail_ax.spines[spine].set_color('red')

    c = thumbnail_ax.scatter(query_results_sample['x'], query_results_sample['y'], c=query_results_sample[column], s=1, cmap='viridis')

    # set lower resolution background map for zoomed out plot of greenland
    ctx.add_basemap(thumbnail_ax, source=ctx.providers.Esri.WorldImagery, zoom=5, crs=geospatial_projection, attribution=False)

t = plt.suptitle('Coverage of CryoTEMPO-EOLIS point product over the Jakobshavn glacier', fontsize=32)

<strong> Next, we can calculate some data quality statistics. </strong>

Firstly we calculate the mean and median uncertainties, as well as the standard deviation.

In [40]:
mean_unc = np.nanmean(point_product_data['uncertainty'])
med_unc = np.nanmedian(point_product_data['uncertainty'])
std_unc = np.std(point_product_data['uncertainty'])

We next set up some histogram bins, requiring 40 bins in the uncertainty range.

In [41]:
bins = np.linspace(np.nanmin(point_product_data['uncertainty']),np.nanmax(point_product_data['uncertainty']),40)

Finally, we plot a histogram of the uncertainty values for this point product dataset, and display the mean and median values, as well as the standard deviation of uncertainties.

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(20,10))

a = axs.hist(point_product_data['uncertainty'], bins = bins, facecolor='paleturquoise', edgecolor='k', alpha=0.6)

axs.vlines(mean_unc, 0, 2.5e4, color='k', linestyle='dashed', label='Mean Uncertainty = {:0.2f}'.format(mean_unc))
axs.vlines(med_unc, 0, 2.5e4, color='b', linestyle='dashed', label='Median Uncertainty = {:0.2f}'.format(med_unc))
axs.vlines(mean_unc - std_unc, 0, 2.5e4, color='orange', linestyle='dashed')
axs.vlines(mean_unc + std_unc, 0, 2.5e4, color='orange', linestyle='dashed',label='$\sigma$ = {:0.2f}'.format(std_unc))

axs.set_ylim(0, 5e4)
axs.set_xlabel('Uncertainty [m]')
axs.set_ylabel('Frequency')
axs.set_title('Uncertainty distribution for CryoTEMPO-EOLIS point product')
plt.legend()